# Phân tích bệnh lý tim mạch bằng phương pháp Shape Similarity Analysis

Paper reference:
ImageCHD: A 3D Computed Tomography
Image Dataset for Classification of Congenital
Heart Disease

## Phân loại bệnh
### I. Dạng Phổ biến
1. Thông liên nhĩ (Atrial Septal Defect – ASD) 
    
    Là hiện tượng hình thành 1 hay nhiều lỗ ở vách ngăn tâm nhĩ. Thông thường, tâm nhĩ trái và tâm nhĩ phải của tim người được phân cách nhau bởi một vách ngăn gọi là vách ngăn tâm nhĩ. Vách ngăn này cùng với vách ngăn tâm thất và 2 vách ngăn nhĩ thất tạo nên 4 buồng tim với chức năng và hoạt động khác nhau, đảm bảo quy trình co bóp của tim. Trong đó, tâm nhĩ phải có nhiệm vụ tống máu xuống thất phải, sau đó lên phổi trao đổi oxy. Nhĩ trái bơm máu xuống thất trái, để bơm máu ra động mạch chủ đi nuôi toàn cơ thể. 

    Trong thời kì bào thai, vách liên nhĩ sẽ không đóng kín để cho một phần máu giàu oxy từ nhĩ phải sang nhĩ trái. Điều này giúp cung cấp oxy cho thai nhi khi phổi chưa hoạt động. Sau khi trẻ được sinh ra, vách ngăn này sẽ đóng lại do sức cản phổi giảm. Lúc này máu lên phổi và trở về nhĩ trái làm cân bằng áp lực hai bên tâm nhĩ. Nhưng nếu vách ngăn này không được đóng kín sẽ gây ra bệnh thông liên nhĩ.

    Theo các nghiên cứu, hầu hết các trường hợp thông liên nhĩ thứ phát với đường kính nhỏ hơn 8mm có thể tự đóng được trong 2-5 năm đầu. Các thể thông liên nhĩ khác không thể tự đóng được, gây ra tình trạng chảy máu từ tâm nhĩ này sang tâm nhĩ khác. Các lỗ thông lớn sẽ gây ra một số biến chứng như: suy tim phải, tăng áp phổi, rối loạn nhịp tim, đột quỵ não do tắc mạch, Đau đầu Migraine, hội chứng Eissenmenger,...

2. Kênh nhĩ thất (Atrioventricular Septal Defect - AVSD) 

    Là một tổ hợp các vấn đề về tim ảnh hưởng đến các vách ngăn giữa các buồng tim và các van giữa các buồng tim trên và dưới. Tên gọi khác là khuyết tật kênh nhĩ thất. Bệnh lý này là bẩm sinh, nghĩa là trẻ sinh ra đã có. Nếu không được điều trị, AVSD có thể gây ra nhiều vấn đề liên quan đến tim và phổi.

3. Thông liên thất (Ventricle Septal Defect - VSD)

    Là một dị tật của vách liên thất, là vách ngăn giữa hai buồng tâm thất của tim. Vách liên thất là một cấu trúc phức tạp gồm: phần cơ, phần màng, phần phễu, phần buồng nhận. Thông thường khi trẻ sinh ra, vách này không có lỗ thông, vì vậy không cho phép máu của hai tâm thất hòa trộn với nhau.

    Triệu chứng: Da, môi và móng tay xanh tím, ăn kém, chậm lớn, thở nhanh hoặc khó thở, mệt mỏi, sưng phù chân, bàn chân hoặc bụng, tim đập nhanh. Đôi khi dị tật vách liên thất không được phát hiện cho đến khi tới tuổi trưởng thành và phát triển các triệu chứng của suy tim như khó thở.

4. Còn ống động mạch (Patent Ductus Arteriosus - PDA)

    Là bệnh lý tim bẩm sinh thường được phát hiện ở trẻ sơ sinh. Ống động mạch nhỏ có thể tự đóng. Đối với những ống động mạch lớn gây ra các vấn đề về lưu thông máu ở trẻ. Máu sẽ lưu thông trực tiếp từ động mạch chủ qua động mạch phổi, dẫn đến gia tăng dòng máu vào hệ tuần hoàn phổi, tăng lượng máu trở về tim trái. Nếu còn ống động mạch lớn, áp lực trong mạch máu phổi cũng tăng theo. Hệ quả là trẻ có nguy cơ bị suy tim khi chỉ mới vài tuần tuổi

    Ống động mạch lớn không được điều trị có thể khiến dòng máu chảy bất thường từ các động mạch lớn trong tim, tăng áp lực trong buồng tim, làm suy yếu cơ tim và gây ra các biến chứng khác.

5. Hẹp Động Mạch Phổi (Pulmonary Atresia - PuA)

    Là một khuyết tật bẩm sinh của van phổi – van kiểm soát lưu lượng máu từ tâm thất phải (buồng dưới bên phải của tim) đến động mạch phổi chính (mạch máu đưa máu từ tim đến phổi).Bệnh hẹp van động mạch phổi là một khiếm khuyết tim hiếm gặp, xảy ra với tần suất ngang nhau giữa các bé trai và bé gái. Tình trạng này thường liên quan đến dị tật tim bẩm sinh được gọi là tứ chứng Fallot.

    Một em bé bị hẹp động mạch phổi có thể cần phẫu thuật hoặc các thủ tục khác ngay sau khi sinh. Vì vậy, khuyết tật bẩm sinh này được coi là một khuyết tật tim bẩm sinh nghiêm trọng (Congenital Heart Disease nghiêm trọng).

6. Hẹp eo động mạch chủ (Coarctation of the Aorta - CA)

    Là một tình trạng bẩm sinh trong đó động mạch chủ bị hẹp, thường là ở khu vực ống động mạch (dây chằng động mạch) chèn vào. Co thắt phổ biến nhất ở vòm động mạch chủ (Aortic Arch). Vòm có thể nhỏ ở trẻ sơ sinh bị hẹp eo. Khi bệnh nhân bị hẹp eo, tâm thất trái làm việc nhiều hơn. Vì động mạch chủ bị thu hẹp nên tâm thất trái phải tạo ra áp lực cao hơn nhiều so với bình thường để ép đủ máu qua động mạch chủ để đưa máu đến phần dưới của cơ thể. Nếu tình trạng thu hẹp nghiêm trọng, tâm thất trái có thể không đủ mạnh để đẩy máu qua chỗ thắt, dẫn đến thiếu máu đến nửa dưới của cơ thể.

7. Bệnh đảo gốc động mạch (Transposition of the Great Arteries - TGA) 

    Là sự thay đổi vị trí của các động mạch lớn (TGA), khi các động mạch “lớn”, động mạch chủ và tâm thất phải, bị đảo ngược về phần gốc từ tim. Động mạch chủ được kết nối với tâm thất phải và động mạch phổi được kết nối với tâm thất trái - hoàn toàn trái ngược với giải phẫu của tim bình thường. 
    Khi các động mạch này đảo ngược, máu ít oxy (màu xanh lam) từ cơ thể trở về tâm nhĩ phải, đi vào tâm thất phải, sau đó đi vào động mạch chủ và trở lại cơ thể. Máu giàu oxy (màu đỏ) trở lại tâm nhĩ trái từ phổi và đi vào tâm thất trái, bơm máu trở lại phổi - ngược lại với cách máu lưu thông bình thường.

8. Tứ chứng Fallot (Tetralogy of Fallot)

   Là một tình trạng hiếm gặp do sự kết hợp của bốn khuyết tật ở tim xuất hiện ngay khi trẻ được sinh ra. Những khiếm khuyết này ảnh hưởng đến cấu trúc của tim, khiến máu nghèo oxy được tim co bóp đi đến khắp mọi cơ quan của cơ thể. Trẻ sơ sinh và trẻ nhỏ bị tứ chứng Fallot thường có da màu xanh tím do máu không mang đủ oxy.
   
   Tứ chứng Fallot thường được chẩn đoán trong giai đoạn bào thai hoặc ngay sau khi sinh. Tuy nhiên, có những trường hợp tứ chứng Fallot không được phát hiện cho đến khi tuổi trưởng thành tùy thuộc vào mức độ nghiêm trọng của các khiếm khuyết và triệu chứng.

### II. Dạng ít phổ biến hơn

9.  Vòng thắt động mạch phổi (Pulmonary Artery Sling - PAS)

    Là một bệnh lý hiếm gặp, được xác định là một dạng vòng mạch máu phổi. Bệnh này gây ra sự chèn ép khí quản do vòng thắt mạch máu phổi bị bóp nghẹt. Bệnh này thường được phát hiện ở trẻ em và có thể gây ra các triệu chứng như khò khè, thở khò khè, ho, thở nhanh và khó thở. Để điều trị bệnh này, các bác sĩ có thể sử dụng phẫu thuật tái tạo khí quản hoặc đặt một ống nong động mạch chủ vào khí quản để giúp nong khí quản

10. Hai đường ra thất phải (Double Outlet Right Ventricle - DORV)

    Là một bệnh tim bẩm sinh hiếm gặp, trong đó động mạch chủ và động mạch phổi không nối với vị trí thông thường trong tim. Trong trái tim bình thường, động mạch chủ kết nối với buồng tim trái dưới. Động mạch phổi kết nối với buồng tim phải dưới. Tuy nhiên, ở trẻ em bị DORV, cả động mạch chủ và động mạch phổi kết nối một phần hoặc hoàn toàn với buồng tim phải dưới. Trẻ em bị DORV cũng có lỗ giữa hai buồng tim dưới. Lỗ này được gọi là khuyết tán giữa hai buồng tim. Lỗ này gây ra sự trộn lẫn giữa máu giàu oxy và máu nghèo oxy. Trẻ em bị bệnh này có thể không đủ oxy trong tuần hoàn máu. Da của họ có thể trở nên xám hoặc xanh. Nếu quá nhiều máu chảy qua động mạch phổi đến phổi, nó có thể dẫn đến suy tim và tăng nguy cơ suy dinh dưỡng.     

11. Thân động mạch chung (Common Arterial Trunk - CAT or Truncus Arteriosus - TA)

    Là một dạng bệnh lý tim mạch bẩm sinh, được đặc trưng bởi việc có một thân động mạch chung duy nhất, thay vì một động mạch chủ và một động mạch phổi chính riêng biệt 1. Thân động mạch chung này mang máu từ tim đến cơ thể, phổi và tim chính nó - nghĩa là, thân động mạch chung này cho ra tuần hoàn hệ thống, phổi và tuần hoàn động mạch cơ tim. Bệnh lý này thường đi kèm với khuyết tật vách liên thất. 

12. Vòm động mạch chủ đôi (Double Aortic Arch - DAA)

    Vòm động mạch chủ đôi là một dị tật tim mạch bẩm sinh tương đối hiếm gặp. DAA là một dị thường của vòm động mạch chủ, trong đó hai vòm động mạch chủ tạo thành một vòng mạch hoàn chỉnh có thể chèn ép khí quản và/hoặc thực quản

13. Bất thường đường thoát tĩnh mạch phổi (Anomalous pulmonary venous connection - APVC)

    Là một khuyết tật tim bẩm sinh tím tái hiếm gặp, trong đó cả bốn tĩnh mạch phổi đều bị đặt sai vị trí và tạo ra các kết nối bất thường với tuần hoàn tĩnh mạch hệ thống. Bệnh được đặc trưng việc một hoặc nhiều tĩnh mạch phổi dẫn máu ra ngoài thất trái của tim thay vì dẫn máu trực tiếp vào thất trái  (Thông thường, các tĩnh mạch phổi đưa máu đã được oxy hóa từ phổi về tâm nhĩ trái, nơi máu có thể được bơm đến phần còn lại của cơ thể).

14. Hẹp động mạch chủ (Aortic Arch Hypoplasia - AAH) 

    Là một bệnh lý tim mạch bẩm sinh, được đặc trưng bởi sự hẹp lại của động mạch chủ trong vùng cổ và ngực. Bệnh lý này thường đi kèm với các bệnh tim mạch bẩm sinh khác và có tác động đáng kể đến việc quản lý bệnh nhân. Một tình trạng tương tự như hẹp eo động mạch chủ, AAH được cho là xuất hiện khi có tắc nghẽn ở một vị trí nhất định trong động mạch chủ. Không giống như hẹp eo, phần lớn động mạch chủ bị tắc nghẽn và do đó tình trạng ngày càng nghiêm trọng hơn, đòi hỏi phải phẫu thuật lâu hơn và khó khắc phục hơn.


15. Gián đoạn vòm động mạch chủ (Interrupted Aortic Arch - IAA)

    Là một tình trạng hiếm gặp trong đó động mạch chủ – mạch máu lớn phát sinh từ tim đưa máu đến phần còn lại của cơ thể – không hoàn chỉnh hoặc bị gián đoạn. Điều này ngăn cản máu giàu oxy đến phần còn lại của cơ thể. Tình trạng này được điều trị bằng phẫu thuật.

16. Tĩnh mạch chủ kép (Double Superior Vena Cava - DSCV)

    Là một biến thể giải phẫu hiếm gặp xuất phát từ tĩnh mạch chủ trên bên trái dai dẳng, do sự thoái lui của tĩnh mạch chính trước về mặt phôi thai đã thất bại. 
    
    SVC bên trái dẫn lưu vào tâm nhĩ phải trong 90% trường hợp qua xoang vành giãn, các vị trí thay thế bao gồm tĩnh mạch chủ dưới, tĩnh mạch gan và tâm nhĩ trái. Sự dẫn lưu SVC trái không điển hình vào tâm nhĩ trái dẫn đến shunt phải sang trái, có thể gây tím tái và có liên quan đến các trường hợp tắc mạch nhiễm trùng.

    SVC bên phải được thấy trong 82-90% trường hợp SVC kép. Sự vắng mặt của SVC phải hoặc SVC trái dẫn lưu vào tâm nhĩ trái có liên quan đến việc tăng tỷ lệ mắc các bệnh tim bẩm sinh như ASD, VSD và TOF.

Nguồn: 

[1] https://www.vinmec.com/vi/tin-tuc/thong-tin-suc-khoe/suc-khoe-tong-quat/trieu-chung-khiem-khuyet-tam-nhi/

[2] https://wellcare.vn/benh/kenh-nhi-that-o-tre-em-phan-loai-trieu-chung-cach-dieu-tri

[3] https://bcare.vn/benh/thong-lien-that-3253.html

[4] https://tamanhhospital.vn/con-ong-dong-mach/

[5] https://mamamy.vn/mang-thai/suc-khoe-thai-nhi/hep-dong-mach-phoi.html

[6] https://en.wikipedia.org/wiki/Coarctation_of_the_aorta

[7] https://wellcare.vn/benh/dao-goc-dong-mach-tren-duoi

[8] https://www.vinmec.com/vi/benh/tu-chung-fallot-3203/

[9] https://www.ncbi.nlm.nih.gov/books/NBK562199/

[10] https://my.clevelandclinic.org/health/diseases/14733-double-outlet-right-ventricle

[11] https://www.cdc.gov/ncbddd/birthdefects/surveillancemanual/quick-reference-handbook/common-truncus.html

[12] https://en.wikipedia.org/wiki/Double_aortic_arch
 
[13] https://en.wikipedia.org/wiki/Anomalous_pulmonary_venous_connection

[14] https://www.rchsd.org/programs-services/heart-institute/services-conditions-treated/conditions-treated/hypoplastic-or-interrupted-aortic-arch/

[15] https://my.clevelandclinic.org/health/diseases/14734-interrupted-aortic-arch

[16] https://radiopaedia.org/cases/double-superior-vena-cava-5#:~:text=Double%20SVC%20is%20a%20rare%20anatomical%20variant%20that,failure%20of%20regression%20of%20the%20anterior%20cardinal%20vein.

[*] https://cand.com.vn/Xa-hoi/BV-Nhi-dong-1---TP-HCM-cuu-song-mot-ca-benh-hiem-gap-i167105/

## Quy trình
1. Đọc và phân tích dữ liệu.
2. Phân chia ra các vùng.
3. Dựng 3D để quan sát bệnh lý.
4. Gán nhãn bệnh lý khi bác sĩ và kỹ thuật viên chẩn đoán hình ảnh đồng ý.
5. Thực hiện train DeepNeural cho phương pháp Shape Similarity Analysis

#### Import các thư viện cần thiết

In [54]:
import os
from supporters import *
import SimpleITK as sitk
import nibabel as nib
import skimage.transform as skTrans

#### Quản lý tên dữ liệu

In [ ]:
def RenameFiles(folder_path):
    files = os.listdir(folder_path)
    file_numbers = [int(file.split('_')[1]) for file in files]
    file_number_pairs = list(zip(files, file_numbers))
    file_number_pairs.sort(key=lambda x: x[1])

    # Rename the files in the correct order
    for i, (old_name, _) in enumerate(file_number_pairs, start=1):
        new_name = f"ct_{i:04d}_label.nii.gz"
        old_path = os.path.join(folder_path, old_name)
        new_path = os.path.join(folder_path, new_name)
        os.rename(old_path, new_path)
        print(f"Renamed: {old_name} to {new_name}")

folder_TrainData = "../../data/VnRawData/TrainData/"
folder_SegmentationData = "../../data/VnRawData/SegmentationData/"

RenameFiles(folder_TrainData)
RenameFiles(folder_SegmentationData)

#### Phân vùng dữ liệu

In [81]:
import os

# Set the path where you want to create the folders
base_path = "../../data/VnRawData/SegmentationData/"

# Create empty folders
for i in range(1, 109):
    folder_name = f"ct_{i:04d}_label"
    folder_path = os.path.join(base_path, folder_name)
    
    # Check if the folder doesn't exist, then create it
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"Created folder: {folder_path}")
    else:
        print(f"Folder already exists: {folder_path}")


Created folder: ../../data/VnRawData/SegmentationData/ct_0001_label
Created folder: ../../data/VnRawData/SegmentationData/ct_0002_label
Created folder: ../../data/VnRawData/SegmentationData/ct_0003_label
Created folder: ../../data/VnRawData/SegmentationData/ct_0004_label
Created folder: ../../data/VnRawData/SegmentationData/ct_0005_label
Created folder: ../../data/VnRawData/SegmentationData/ct_0006_label
Created folder: ../../data/VnRawData/SegmentationData/ct_0007_label
Created folder: ../../data/VnRawData/SegmentationData/ct_0008_label
Created folder: ../../data/VnRawData/SegmentationData/ct_0009_label
Created folder: ../../data/VnRawData/SegmentationData/ct_0010_label
Created folder: ../../data/VnRawData/SegmentationData/ct_0011_label
Created folder: ../../data/VnRawData/SegmentationData/ct_0012_label
Created folder: ../../data/VnRawData/SegmentationData/ct_0013_label
Created folder: ../../data/VnRawData/SegmentationData/ct_0014_label
Created folder: ../../data/VnRawData/Segmentatio

In [82]:
def SaveImage(image, file_path_save):
    converted_array = np.array(image, dtype=np.float32)
    converted_array = np.transpose(converted_array, (2, 1, 0))
    affine = np.eye(4)
    nifti_file = nib.Nifti1Image(converted_array, affine)
    nib.save(nifti_file, file_path_save)
    print(f"{file_path_save} successfully")

def Seperation(img, file_path_save):
    array = img.flatten()
    unique_values, counts = np.unique(array, return_counts=True)
    
    for index, value in enumerate(unique_values): 
        label_array = np.copy(img) 
        label_array[np.where(label_array != value)] = 0 
        SaveImage(label_array, file_path_save + f'_{index + 1}.nii.gz')

def Segmentation(folder_path):
    files = os.listdir(folder_path)
    for file in files:
        file_path = os.path.join(folder_path, file)
        img_raw = sitk.ReadImage(file_path, sitk.sitkFloat32)
        img = sitk.GetArrayFromImage(img_raw)
        
        file_path_save = file[:13] + '/' + file[:13]
        file_path_save = os.path.join(folder_path, file_path_save)
        
        Seperation(img, file_path_save)
        
     
Segmentation(
    folder_path='../../data/VnRawData/SegmentationData/'
)

../../data/VnRawData/SegmentationData/ct_0001_label/ct_0001_label_1.nii.gz successfully
../../data/VnRawData/SegmentationData/ct_0001_label/ct_0001_label_2.nii.gz successfully
../../data/VnRawData/SegmentationData/ct_0001_label/ct_0001_label_3.nii.gz successfully
../../data/VnRawData/SegmentationData/ct_0001_label/ct_0001_label_4.nii.gz successfully
../../data/VnRawData/SegmentationData/ct_0001_label/ct_0001_label_5.nii.gz successfully
../../data/VnRawData/SegmentationData/ct_0001_label/ct_0001_label_6.nii.gz successfully
../../data/VnRawData/SegmentationData/ct_0001_label/ct_0001_label_7.nii.gz successfully
../../data/VnRawData/SegmentationData/ct_0001_label/ct_0001_label_8.nii.gz successfully
../../data/VnRawData/SegmentationData/ct_0001_label/ct_0001_label_1.nii.gz successfully
../../data/VnRawData/SegmentationData/ct_0001_label/ct_0001_label_2.nii.gz successfully
../../data/VnRawData/SegmentationData/ct_0001_label/ct_0001_label_3.nii.gz successfully
../../data/VnRawData/Segmentatio

#### Xử lý dữ liệu

In [83]:
import os

# Set the path where you want to create the folders
base_path = "../../data/VnRawData/SegmentationData/"

# Create empty folders
for i in range(1, 109):
    folder_name = f"ct_{i:04d}_label_resized"
    folder_path = os.path.join(base_path, folder_name)
    
    # Check if the folder doesn't exist, then create it
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"Created folder: {folder_path}")
    else:
        print(f"Folder already exists: {folder_path}")


Created folder: ../../data/VnRawData/SegmentationData/ct_0001_label_resized
Created folder: ../../data/VnRawData/SegmentationData/ct_0002_label_resized
Created folder: ../../data/VnRawData/SegmentationData/ct_0003_label_resized
Created folder: ../../data/VnRawData/SegmentationData/ct_0004_label_resized
Created folder: ../../data/VnRawData/SegmentationData/ct_0005_label_resized
Created folder: ../../data/VnRawData/SegmentationData/ct_0006_label_resized
Created folder: ../../data/VnRawData/SegmentationData/ct_0007_label_resized
Created folder: ../../data/VnRawData/SegmentationData/ct_0008_label_resized
Created folder: ../../data/VnRawData/SegmentationData/ct_0009_label_resized
Created folder: ../../data/VnRawData/SegmentationData/ct_0010_label_resized
Created folder: ../../data/VnRawData/SegmentationData/ct_0011_label_resized
Created folder: ../../data/VnRawData/SegmentationData/ct_0012_label_resized
Created folder: ../../data/VnRawData/SegmentationData/ct_0013_label_resized
Created fold

In [88]:
def ResizeImage(image, x_, y_,z_):
    return np.array(skTrans.resize(image, (x_, y_, z_), order=1, preserve_range=True, anti_aliasing=False), dtype="float16")

def SaveImage(image, file_path_save):
    converted_array = np.array(image, dtype=np.float32)
    converted_array = np.transpose(converted_array, (2, 1, 0))
    affine = np.eye(4)
    nifti_file = nib.Nifti1Image(converted_array, affine)
    nib.save(nifti_file, file_path_save)

def Preprocessing(folder_path, folder_save):
    files = os.listdir(folder_path)
    for file in files:
        file_path = os.path.join(folder_path, file)
        print(file_path)
        img_raw = sitk.ReadImage(file_path, sitk.sitkFloat32)
        img = sitk.GetArrayFromImage(img_raw)
        img_resized = ResizeImage(img, 512, 600, 600)
        file_path_save = os.path.join(folder_save, file)
        SaveImage(img_resized, file_path_save)
        
    
main_folder_path = "../../data/VnRawData/SegmentationData/"
subfolders = [f for f in os.listdir(main_folder_path) if os.path.isdir(os.path.join(main_folder_path, f)) and "resized" not in f]
resize_subfolders = [f for f in os.listdir(main_folder_path) if os.path.isdir(os.path.join(main_folder_path, f)) and "resized" in f]

for subfolder, resize_subfolder in zip(subfolders, resize_subfolders):
    subfolder_path = os.path.join(main_folder_path, subfolder)
    resize_subfolder_path = os.path.join(main_folder_path, resize_subfolder)
    
    Preprocessing(
        folder_path=subfolder_path,
        folder_save=resize_subfolder_path
    )

../../data/VnRawData/SegmentationData/ct_0001_label\ct_0001_label_1.nii.gz
../../data/VnRawData/SegmentationData/ct_0001_label\ct_0001_label_2.nii.gz
../../data/VnRawData/SegmentationData/ct_0001_label\ct_0001_label_3.nii.gz
../../data/VnRawData/SegmentationData/ct_0001_label\ct_0001_label_4.nii.gz
../../data/VnRawData/SegmentationData/ct_0001_label\ct_0001_label_5.nii.gz
../../data/VnRawData/SegmentationData/ct_0001_label\ct_0001_label_6.nii.gz
../../data/VnRawData/SegmentationData/ct_0001_label\ct_0001_label_7.nii.gz
../../data/VnRawData/SegmentationData/ct_0001_label\ct_0001_label_8.nii.gz
../../data/VnRawData/SegmentationData/ct_0002_label\ct_0002_label_1.nii.gz
../../data/VnRawData/SegmentationData/ct_0002_label\ct_0002_label_2.nii.gz
../../data/VnRawData/SegmentationData/ct_0002_label\ct_0002_label_3.nii.gz
../../data/VnRawData/SegmentationData/ct_0002_label\ct_0002_label_4.nii.gz
../../data/VnRawData/SegmentationData/ct_0002_label\ct_0002_label_5.nii.gz
../../data/VnRawData/Segm